## Objective- Join tables togeather with Big Query sql query and clean for EDA.

### Steps -

1 - Querys multiple BQ tables with join.  

2 - check for nulls and duplicates and clean query result. 

3- save cleaned data.  



In [13]:
import os
import pandas as pd
import numpy as np
import pandas_gbq
from google.oauth2 import service_account
from google.cloud import bigquery
from config import creds

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= creds



credentials = service_account.Credentials.from_service_account_file(creds)



project_id = 'san-jose-earthquakes-dev'

client = bigquery.Client(project_id)

# 1 - query all merged features from fb_ref tables in BQ

In [6]:
# updated with def stats blocks 
sql = """
WITH
  -- opposing team stats
  opp AS (
  SELECT
    Squad AS Squad_from_op,
    Opposing_Poss,
    Opposing_Touches
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.opposing_possesion` ),
  -- Mich with aerial stats
  mish AS (
  SELECT
    Player as mish_player,
    Yellow_cards,
    Red_cards,
    Second_yellow,
    Fouls,
    Fouls_drawn,
    Offsides,
    Crosses,
    Interceptions as def_interceptions,
    Tackles_won as from_mish_tackles_won,
    PK_won,
    PK_conceded,
    OG,
    Recoveries,
    Aerial_Duels_Won,
     Aerial_Duels_lost,
    Aerial_win_pct 
    from 
    `san-jose-earthquakes-dev.MLS_2021_fbref.Miscellaneous_stats` ),
  -- defensive stats
  def AS (
  SELECT
    *
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.Defensive_stats` ),
  -- player passing stats
  pass AS (
  SELECT
    Player,
    fb_id,
    player_name,
    Nineties,
    CAST(xA AS float64) AS xA,
    CAST(Key_Pass AS int64) AS Key_pass,
    CAST(Comp_prog AS float64) AS Comp_prog,
    Total_Cmp,
    Total_Att,
    Total_Cmp_pct,
    Total_Tot_Dist,
    Total_Prg_Dist,
    Short_Cmp,
    Short_Att,
    Short_Cmp_pct,
    Medium_Cmp,
    Medium_Att,
    Medium_Cmp_pct,
    Long_Cmp,
    Long_Att,
    Long_Cmp_pct,
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.passing_stats` ),
  --player possesion stats
  posses AS(
  SELECT
    *
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.possesion_stats` )
  --
  ### main query below
  --
SELECT
  pass.fb_id,
  pass.player_name,
  opp.Squad_from_op,
  opp.Opposing_Poss,
  opp.Opposing_Touches,
  -- def stats
  
  def.Player,
  def.Nation,
  def.Position,

  def.Total_tackles,
  def.Tackles_Won,	
  def.Tackles_Def_3rd,	
  def.Tackles_Mid_3rd,	
  def.Tackles_Att_3rd,	
  def.Num_Dribblers_tackled,	
  def.Num_Dribbled_past,	
  def.total_pressing,	
  def.Won_ball_5_sec,	
  def.pct_won,	
  def.pres_Def_3rd,	
  def.pres_Mid_3rd,	
  def.pres_Att_3rd,	
  def.Blocks,	
  def.Blocked_shots,	
  def.Blocked_SOT,	
  def.Blocked_pass,	
  def.Interceptions,	
  def.Tackles_and_Ints,	
  def.Clearences,
  pass.Nineties,
  pass.xA,
  pass.Key_pass,
  pass.Comp_prog,
  pass.Total_Cmp,
  pass.Total_Att,
  pass.Total_Cmp_pct,
  pass.Total_Tot_Dist,
  pass.Total_Prg_Dist,
  pass.Short_Cmp,
  pass.Short_Att,
  pass.Short_Cmp_pct,
  pass.Medium_Cmp,
  pass.Medium_Att,
  pass.Medium_Cmp_pct,
  pass.Long_Cmp,
  pass.Long_Att,
  pass.Long_Cmp_pct,
  -- posses below
  posses.touches,
  posses.touches_def_box,
  posses.touches_def_3rd,
  posses.touches_mid_3rd,
  posses.touches_att_3rd,
  posses.touches_att_box,
  posses.touch_live,
  posses.dribble_Succ,
  posses.dribble_Att,
  posses.dribble_Succ_pct,
  posses.num_dribble_past,
  posses.dribble_megs,
  posses.carries,
  posses.carries_dist,
  posses.carries_prog_dist,
  posses.carries_prog,
  posses.carries_att_3rd,
  posses.carries_att_box,
  posses.miss_control,
  posses.dispossed,
  posses.recep_targ,
  posses.recep_succ,
  posses.recept_pct,
  posses.prog_pass_rec

  -- mish

  Yellow_cards,
    Red_cards,
    Second_yellow,
    Fouls,
    Fouls_drawn,
    Offsides,
    Crosses,
    def_interceptions,
    from_mish_tackles_won,
    PK_won,
    PK_conceded,
    OG,
    Recoveries,
    Aerial_Duels_Won Aerial_Duels_lost,
    Aerial_win_pct 
FROM
  def
LEFT JOIN
  pass
ON
  def.Player = pass.Player
left join 
  mish 
on 
  def.Player = mish.mish_player
LEFT JOIN
  posses
ON
  def.Player = posses.Player
LEFT JOIN
  opp
ON
  def.Team = opp.Squad_from_op
ORDER BY
  Key_pass DESC

"""
df = pandas_gbq.read_gbq(sql,project_id=project_id)



Downloading: 100%|██████████| 925/925 [00:01<00:00, 681.86rows/s]


In [7]:
info= pd.DataFrame({"values":{col:df[col].unique() for col in df},
              'type':{col:df[col].dtype for col in df},
              'unique values':{col:len(df[col].unique()) for col in df},
              'NA values':{col:str(round(sum(df[col].isna())/len(df),2))+'%' for col in df},
              'Duplicated Values':{col:sum(df[col].duplicated()) for col in df}
             })

info.T

,fb_id,player_name,Squad_from_op,Opposing_Poss,Opposing_Touches,Player,Nation,Position,Total_tackles,Tackles_Won,...,Offsides,Crosses,def_interceptions,from_mish_tackles_won,PK_won,PK_conceded,OG,Recoveries,Aerial_Duels_lost,Aerial_win_pct
values,"[11c30a46, acd47bc0, 614b26e3, 0adeafaa, f99f3...","[Carles Gil, Julian Gressel, Emanuel Reynoso, ...","[New England, D.C. United, Minnesota Utd, Real...","[49.1, 54.0, 49.9, 46.5, 51.0, 48.3, 47.1, 42....","[20024, 20526, 20887, 19279, 19332, 20501, 193...","[Carles Gil\11c30a46, Julian Gressel\acd47bc0,...","[es ESP, de GER, ar ARG, sk SVK, am ARM, us US...","[MF, DF, MFFW, FWMF, FW, FWDF, DFMF, MFDF, DFF...","[10, 46, 59, 24, 48, 56, 32, 35, 47, 54, 90, 2...","[4, 19, 43, 16, 26, 42, 20, 27, 39, 65, 24, 21...",...,"[0, 4, 2, 1, 5, 8, 3, 14, 26, 10, 7, 6, 22, 15...","[114, 144, 69, 46, 70, 59, 100, 62, 126, 41, 5...","[23, 68, 27, 25, 28, 30, 44, 53, 40, 21, 18, 3...","[4, 19, 43, 16, 26, 42, 20, 27, 39, 65, 24, 21...","[0, 1, 3, 2]","[0, 1, 2, 3]","[0, 1, 2]","[164, 251, 198, 203, 167, 178, 153, 163, 243, ...","[4, 30, 18, 10, 5, 6, 22, 26, 19, 8, 17, 15, 1...","[17.4, 44.8, 28.6, 12.9, 23.3, 27.8, 13.6, 44...."
type,object,object,object,float64,Int64,object,object,object,object,object,...,object,object,object,object,object,object,object,object,object,object
unique values,730,730,27,24,27,730,85,10,88,63,...,23,82,80,63,4,4,3,241,86,285
NA values,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,...,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.11%
Duplicated Values,195,195,898,901,898,195,840,915,837,862,...,902,843,845,862,921,921,922,684,839,640


In [8]:
df.describe()

,Opposing_Poss,Opposing_Touches,xA,Key_pass,Comp_prog
count,925.000000,925.000000,925.000000,925.000000,925.000000
mean,50.083027,20247.580541,1.079351,10.406486,37.052973
std,3.152554,1037.474638,1.569396,14.663216,41.359008
min,42.600000,17449.000000,0.000000,0.000000,0.000000
25%,48.300000,19650.000000,0.100000,1.000000,5.000000
50%,50.200000,20480.000000,0.400000,5.000000,21.000000
75%,51.900000,21096.000000,1.500000,14.000000,55.000000
max,55.100000,22416.000000,12.200000,134.000000,277.000000


In [9]:
df.fb_id.nunique()

730

In [10]:
df.duplicated(subset=['fb_id']).sum()

195

In [11]:
df.duplicated(subset=['fb_id']).value_counts()

False    730
True     195
dtype: int64

In [12]:
# Considering certain columns for dropping duplicates
df.drop_duplicates(subset=['player_name'],keep='first',inplace=True)
df.duplicated(subset=['fb_id']).value_counts()

False    730
dtype: int64

look for null values in dataframe

In [11]:
## change column to clarify tackle pct
df.rename(columns={'pct_won':'Tackle_pct_won'},inplace=True)
missing = [df.isnull().sum(),100*df.isnull().mean()]

# check for missing values
m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()

,count,%
dribble_Succ_pct,106.0,14.520548
Aerial_win_pct,90.0,12.328767
Long_Cmp_pct,24.0,3.287671
Tackle_pct_won,16.0,2.191781
Short_Cmp_pct,5.0,0.684932


## Decision - drop players with nulls in features what are less than 2% missing 



In [13]:
df.dropna(subset=['Long_Cmp_pct','Short_Cmp_pct','Medium_Cmp_pct','Total_Cmp_pct','recept_pct'],inplace=True )
missing = [df.isnull().sum(),100*df.isnull().mean()]

m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()

m = pd.DataFrame(missing)
m = m.T
m.columns=['count','%']
m =m.sort_values(by='count',ascending=False)
m.head(10)

,count,%
dribble_Succ_pct,91.0,12.907801
Aerial_win_pct,80.0,11.347518
Tackle_pct_won,15.0,2.127660
dribble_Att,0.0,0.000000
carries_prog,0.0,0.000000
carries_prog_dist,0.0,0.000000
carries_dist,0.0,0.000000
carries,0.0,0.000000
dribble_megs,0.0,0.000000
num_dribble_past,0.0,0.000000


In [16]:
df[['dribble_Succ_pct','Aerial_win_pct']].describe()

,dribble_Succ_pct,Aerial_win_pct
count,614,625
unique,197,283
top,100,50
freq,61,28


In [17]:
## insert mean value for columns below 

df[['dribble_Succ_pct','Aerial_win_pct','Tackle_pct_won']] = df[['dribble_Succ_pct','Aerial_win_pct','Tackle_pct_won']].astype(float)

df['Tackle_pct_won'].fillna((df['Tackle_pct_won'].mean()), inplace=True)
df['dribble_Succ_pct'].fillna((df['dribble_Succ_pct'].mean()), inplace=True)
df['Aerial_win_pct'].fillna((df['Aerial_win_pct'].mean()), inplace=True)

## Add fb_ref shooting data to df

In [18]:
path = '../data/2021_player_shooting.csv'
dfs= pd.read_csv(path)
dfs.columns = dfs.iloc[0]
dfs.drop(0,inplace=True)
dfs.drop(columns=['Pos','Rk','Born','Age','90s','Squad','Nation','Matches',],inplace=True)
dfs.head()

,Player,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG
1,Paxten Aaronson\4cd41883,3,11,9,81.8,1.66,1.35,0.27,0.33,16.8,0,0,0,1.3,1.3,0.12,+1.7,+1.7
2,Saad Abdul-Salaam\4705543b,0,3,1,33.3,0.29,0.1,0,0,35.1,0,0,0,0.1,0.1,0.02,-0.1,-0.1
3,Luciano Abecasis\6c510f2d,0,8,2,25,0.58,0.15,0,0,19.2,0,0,0,0.5,0.5,0.07,-0.5,-0.5
4,Ramón Ábila\db5ee72d,3,10,6,60,4,2.4,0.3,0.5,9.6,0,0,0,1,1,0.1,+2.0,+2.0
5,Ramón Ábila\db5ee72d,2,13,5,38.5,4.37,1.68,0.15,0.4,13.8,0,0,1,2.7,1.8,0.14,-0.7,+0.2


In [19]:
dfs.drop_duplicates(subset='Player',inplace=True)
dfs.duplicated(subset='Player').sum()

0

In [20]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 1 to 743
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Player   730 non-null    object
 1   Gls      730 non-null    object
 2   Sh       730 non-null    object
 3   SoT      730 non-null    object
 4   SoT%     598 non-null    object
 5   Sh/90    730 non-null    object
 6   SoT/90   730 non-null    object
 7   G/Sh     598 non-null    object
 8   G/SoT    525 non-null    object
 9   Dist     598 non-null    object
 10  FK       730 non-null    object
 11  PK       730 non-null    object
 12  PKatt    730 non-null    object
 13  xG       730 non-null    object
 14  npxG     730 non-null    object
 15  npxG/Sh  598 non-null    object
 16  G-xG     730 non-null    object
 17  np:G-xG  730 non-null    object
dtypes: object(18)
memory usage: 108.4+ KB


In [21]:
missing = [dfs.isnull().sum(),100*dfs.isnull().mean()]
m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()

,count,%
0,,
G/SoT,205.0,28.082192
Dist,132.0,18.082192
npxG/Sh,132.0,18.082192
SoT%,132.0,18.082192
G/Sh,132.0,18.082192


### Fill missing shooting columns with zero instead of avg as players probably didnt take any shoots and shouldnt be given stat.

In [22]:
cols = ['G/SoT','Dist','SoT%','npxG/Sh','G/Sh']
for col in cols:
    dfs[col] = dfs[col].fillna(0)
missing = [dfs.isnull().sum(),100*dfs.isnull().mean()]
m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()  

,count,%
0,,
Player,0.0,0.0
Gls,0.0,0.0
G-xG,0.0,0.0
npxG/Sh,0.0,0.0
npxG,0.0,0.0


## Join shooting and salary data to larger data set to have all initial features from all stat types to work with

In [23]:
df_concat = pd.merge(df, dfs, left_on='Player', right_on='Player',how='left')
# df_concat.drop(columns=['Unnamed: 0'],inplace=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705 entries, 0 to 704
Columns: 101 entries, fb_id to np:G-xG
dtypes: Int64(2), float64(6), object(93)
memory usage: 563.2+ KB


In [24]:
path = "../data/player_salary.csv"
dfs= pd.read_csv(path)
dfs.shape

(871, 8)

In [26]:
## left join stats to salary on name, special characters prevent for perfect match between spelling 
## in the two data sets. 

dfr = pd.merge(df_concat,dfs,how='left',left_on='player_name',right_on='name',indicator=True)
print(dfr.shape)
dfr['_merge'].value_counts()

(705, 110)


both          521
left_only     184
right_only      0
Name: _merge, dtype: int64

In [27]:
dfr.to_csv('../data/all_salary_stats.csv')

In [36]:
dfr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705 entries, 0 to 919
Data columns (total 84 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fb_id                  705 non-null    object 
 1   player_name            705 non-null    object 
 2   Squad_from_op          705 non-null    object 
 3   Opposing_Poss          705 non-null    float64
 4   Opposing_Touches       705 non-null    Int64  
 5   Player                 705 non-null    object 
 6   Nation                 705 non-null    object 
 7   Position               705 non-null    object 
 8   Total_tackles          705 non-null    object 
 9   Tackles_Won            705 non-null    object 
 10  Tackles_Def_3rd        705 non-null    object 
 11  Tackles_Mid_3rd        705 non-null    object 
 12  Tackles_Att_3rd        705 non-null    object 
 13  Num_Dribblers_tackled  705 non-null    object 
 14  Num_Dribbled_past      705 non-null    object 
 15  total_

use bar graphs to observe distriubtion of key features 